In [53]:
import pandas as pd
import numpy as np
import yfinance as yf
from datetime import datetime, timedelta

def fetch_sp500_tickers():
    """Fetch current S&P 500 tickers from Wikipedia"""
    url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
    tables = pd.read_html(url)
    sp500_table = tables[0]
    tickers = sp500_table['Symbol'].tolist()
    return [ticker.replace(".", "-") for ticker in tickers]
    
# Download S&P 500 data for the Index Filter
def download_sp500_data(start, end):
    sp500 = yf.download("^GSPC", start = start, end = end, interval="1wk")
    # Flatten multi-index column names
    sp500.columns = sp500.columns.get_level_values(0)  
    # Calculate the moving average
    sp500['10_week_MA'] = sp500['Close'].rolling(window=10).mean()
    # Drop NaNs to ensure alignment
    sp500 = sp500.dropna(subset=['10_week_MA'])
    # Now apply the condition
    sp500['Index_Filter'] = np.where(sp500['Close'] > sp500['10_week_MA'], 'GREEN', 'RED')
    sp500.columns = sp500.columns.get_level_values(0)  
    sp500 = sp500.reset_index()
    return sp500

# Download stock data for a given ticker
def download_stock_data(ticker, start, end):
    stock = yf.download(ticker, start = start, end = end, interval="1wk")
    # Flatten column names
    stock.columns = stock.columns.get_level_values(0)  
    # Calculate rolling high and ROC
    stock['20_week_high'] = stock['High'].rolling(window=20).max()  # Use High instead of Close
    stock['20_week_ROC'] = ((stock['Close'] - stock['Close'].shift(20)) / stock['Close'].shift(20)) * 100
    stock.columns = stock.columns.get_level_values(0)  
    stock = stock.reset_index()
    stock.columns = ["Stock_" + col for col in stock.columns]
    return stock
    
# Generate buy signals based on the strategy rules
def generate_signals(stock, sp500):
    # Merge stock data with S&P 500 Index Filter
    merged_df = stock.merge(sp500, left_on="Stock_Date", right_on="Date", how="left")

    # Drop duplicate Date column (keeping Stock_Date)
    merged_df = merged_df.drop(columns=["Date","Close","High", "Low","Volume"])
    merged_df = merged_df.dropna()
    
    # Generate Buy Signal
    merged_df['Buy_Signal'] = np.where((merged_df['Stock_High'] >= merged_df['Stock_20_week_high']) &
        (merged_df['Stock_20_week_ROC'] > 30) &
        (merged_df['Index_Filter'] == 'GREEN'), 1, 0)
    
    # Generate Sell Signal
    merged_df["Close_Change"] = merged_df["Stock_Close"].pct_change()  
    condition_40 = merged_df["Close_Change"] <= -0.40  

    # Condition 2: 10% or more drop when Index_Filter is 'RED'
    condition_10_red = (merged_df["Close_Change"] <= -0.10) & (merged_df["Index_Filter"] == "RED")

    # Apply conditions to create 'Sell_Indicator'
    merged_df["Sell_Indicator"] = (condition_40 | condition_10_red).astype(int)

    
    return merged_df





In [66]:
start = '1995-01-01'
end = '2024-02-07'
sp500 = download_sp500_data(start = start, end = end)
tickers = fetch_sp500_tickers()
df_list = []
for ticker in tickers:
    try:
        print(f"Processing: {ticker}")
        stock = download_stock_data(ticker, start = start, end = end)
        
        if stock.empty:
            logging.warning(f"{ticker}: No data available, skipping.")
            continue  # Skip if no data
        
        df = generate_signals(stock, sp500)
        df_list.append((ticker, df))  # Store with ticker
        
    except Exception as e:
        continue  # Skip this ticker and move to the next one

[*********************100%***********************]  1 of 1 completed


Processing: MMM


[*********************100%***********************]  1 of 1 completed


Processing: AOS


[*********************100%***********************]  1 of 1 completed


...

[*********************100%***********************]  1 of 1 completed


Processing: ZBH


[*********************100%***********************]  1 of 1 completed


Processing: ZTS


[*********************100%***********************]  1 of 1 completed


In [69]:
import logging

# Get the most recent date available in the dataset
latest_date = max(date for _, df in df_list for date in df["Stock_Date"])

# Find tickers that have a Buy Signal on the most recent date
buy_signal_tickers = [
    ticker for ticker, df in df_list 
    if not df[df["Stock_Date"] == latest_date].empty and df[df["Stock_Date"] == latest_date]["Buy_Signal"].iloc[0] == 1
]

# Display the tickers with buy signals
print("Tickers with Buy Signal for the most recent week:", buy_signal_tickers)


Tickers with Buy Signal for the most recent week: ['AXP', 'CDNS', 'COST', 'FAST', 'IDXX', 'LLY', 'MAS', 'PH', 'TGT']


In [68]:
for df,i in df_list:
    print(df)
    print(i)

MMM
     Stock_Date  Stock_Close  Stock_High  Stock_Low  Stock_Open  Stock_Volume  \
20   1995-05-21    10.369263   10.704476  10.324568   10.369263       6706929   
21   1995-05-28    10.458659   10.771525  10.279879   10.436312       4820598   
22   1995-06-04    10.480999   10.659779  10.391609   10.480999       4582355   
23   1995-06-11    10.548041   10.726822  10.503346   10.503346       8107445   
24   1995-06-18    10.503349   10.860910  10.413959   10.548044       8498299   
...         ...          ...         ...        ...         ...           ...   
1514 2024-01-07    87.372787   89.021333  86.637406   87.485924      14569314   
1515 2024-01-14    87.081871   87.566735  84.932297   87.009143      13680446   
1516 2024-01-21    77.570419   88.415253  74.935989   87.679871      61398693   
1517 2024-01-28    76.665344   78.297727  75.913806   77.578508      25016732   
1518 2024-02-04    75.832993   76.536048  74.669311   76.164320      14329514   

      Stock_20_week_hig

In [40]:
import pandas as pd
import numpy as np

# Initialize portfolio
initial_cash = 10000
portfolio_cash = initial_cash
max_positions = 10
portfolio_stocks = {}  # Store {ticker: {shares, buy_price}}

# Create a DataFrame to track portfolio value over time
portfolio_history = []

# Process each week in chronological order
all_dates = sorted(set(date for _, df in df_list for date in df["Stock_Date"]))

for current_date in all_dates:
    # Get stocks available for the current date
    available_stocks = [(ticker, df[df["Stock_Date"] == current_date]) for ticker, df in df_list]
    available_stocks = [(ticker, df) for ticker, df in available_stocks if not df.empty]

    # Create a dictionary for fast lookup
    stock_dict = {ticker: df for ticker, df in available_stocks}

    # SELL Stocks: If "Sell_Indicator" == 1
    for ticker in list(portfolio_stocks.keys()):
        df = stock_dict.get(ticker)
        if df is not None and not df.empty and "Sell_Indicator" in df.columns:
            if df["Sell_Indicator"].iloc[0] == 1:
                sell_price = df["Stock_Close"].iloc[0]
                shares = portfolio_stocks[ticker]["shares"]
                portfolio_cash += shares * sell_price  # Add cash from sale
                del portfolio_stocks[ticker]  # Remove stock

    # Update available cash AFTER selling
    num_positions = max_positions - len(portfolio_stocks)
    available_cash = portfolio_cash / num_positions if num_positions > 0 else 0

    # BUY Stocks: If "Buy_Signal" == 1
    for ticker, df in available_stocks:
        if "Buy_Signal" in df.columns and df["Buy_Signal"].iloc[0] == 1:
            if ticker not in portfolio_stocks and len(portfolio_stocks) < max_positions:
                buy_price = df["Stock_Close"].iloc[0]
                shares = available_cash / buy_price
                portfolio_stocks[ticker] = {"shares": shares, "buy_price": buy_price}
                portfolio_cash -= available_cash  # Reduce cash

    # Calculate total portfolio value
    stock_value = 0
    for ticker, stock in portfolio_stocks.items():
        df = stock_dict.get(ticker)
        if df is not None and not df.empty:
            stock_price = df["Stock_Close"].iloc[0]
        else:
            stock_price = stock["buy_price"]  # Use last known price
        stock_value += stock["shares"] * stock_price
    # Calculate total portfolio value
stock_value = sum(
    stock["shares"] * stock_dict[ticker]["Stock_Close"].iloc[0]
    if ticker in stock_dict else stock["buy_price"]
    for ticker, stock in portfolio_stocks.items()
)

total_portfolio_value = portfolio_cash + stock_value

# Store held stocks as a comma-separated string
held_stocks = ", ".join(portfolio_stocks.keys())

# Append portfolio state
portfolio_history.append({
    "Date": current_date,
    "Portfolio_Value": total_portfolio_value,
    "Held_Stocks": held_stocks  # New column for tracking holdings
})

# Convert history to DataFrame
portfolio_df = pd.DataFrame(portfolio_history)
print(portfolio_df)


    total_portfolio_value = portfolio_cash + stock_value
    portfolio_history.append({"Date": current_date, "Portfolio_Value": total_portfolio_value})

# Convert history to DataFrame
portfolio_df = pd.DataFrame(portfolio_history)
print(portfolio_df)


           Date  Portfolio_Value
0    1995-05-21     10000.000000
1    1995-05-28     10082.661055
2    1995-06-04     10131.865875
3    1995-06-11     10433.206401
4    1995-06-18     11148.271711
...         ...              ...
1541 2024-12-01    839158.375276
1542 2024-12-08    813156.885009
1543 2024-12-15    788932.736156
1544 2024-12-22    796435.108329
1545 2024-12-29    789214.976846

[1546 rows x 2 columns]


In [46]:
# Initialize portfolio
initial_cash = 10000
portfolio_cash = initial_cash
max_positions = 10
portfolio_stocks = {}  # Store {ticker: {shares, buy_price}}
portfolio_history = []


# Process each week in chronological order
all_dates = sorted(set(date for _, df in df_list for date in df["Stock_Date"]))

for current_date in all_dates:
    # Filter stocks that have data for the current date
    available_stocks = [(ticker, df[df["Stock_Date"] == current_date]) for ticker, df in df_list]
    available_stocks = [(ticker, df) for ticker, df in available_stocks if not df.empty]

    # Create a dictionary for fast lookup
    stock_dict = {ticker: df for ticker, df in available_stocks}

    # SELL Stocks: If "Sell_Indicator" == 1, remove from portfolio
    for ticker in list(portfolio_stocks.keys()):
        df = stock_dict.get(ticker)
        if df is not None and df["Sell_Indicator"].iloc[0] == 1:
            sell_price = df["Stock_Close"].iloc[0]
            shares = portfolio_stocks[ticker]["shares"]
            portfolio_cash += shares * sell_price  # Add cash from sale
            del portfolio_stocks[ticker]  # Remove stock

    # Update available cash AFTER selling
    remaining_positions = max_positions - len(portfolio_stocks)
    available_cash = portfolio_cash / remaining_positions if remaining_positions > 0 else portfolio_cash


    # BUY Stocks: If "Buy_Signal" == 1
    for ticker, df in available_stocks:
        if df["Buy_Signal"].iloc[0] == 1 and ticker not in portfolio_stocks and len(portfolio_stocks) < max_positions:
            buy_price = df["Stock_Close"].iloc[0]
            shares = available_cash / buy_price
            portfolio_stocks[ticker] = {"shares": shares, "buy_price": buy_price}
            portfolio_cash -= available_cash  # Reduce cash

    # Calculate total portfolio value
    stock_value = sum(
        stock["shares"] * stock_dict[ticker]["Stock_Close"].iloc[0]
        if ticker in stock_dict else stock["buy_price"]
        for ticker, stock in portfolio_stocks.items()
    )

    total_portfolio_value = portfolio_cash + stock_value

    # Store held stocks as a comma-separated string
    held_stocks = ", ".join(portfolio_stocks.keys()) if portfolio_stocks else "None"

    # Append portfolio state
    portfolio_history.append({
        "Date": current_date,
        "Portfolio_Value": total_portfolio_value,
        "Held_Stocks": held_stocks  # New column for tracking holdings
    })

# Convert history to DataFrame
portfolio_df = pd.DataFrame(portfolio_history)

# Display result
print(portfolio_df)


           Date  Portfolio_Value  \
0    1995-05-21     10000.000000   
1    1995-05-28     10082.659077   
2    1995-06-04     10131.864149   
3    1995-06-11     10433.205832   
4    1995-06-18     11148.269667   
...         ...              ...   
1541 2024-12-01    965830.601109   
1542 2024-12-08    935904.146529   
1543 2024-12-15    908023.326586   
1544 2024-12-22    916658.192866   
1545 2024-12-29    908348.171346   

                                            Held_Stocks  
0     AFL, ADI, AMAT, BK, CSCO, HPQ, INTC, JBL, KLAC...  
1     AFL, ADI, AMAT, BK, CSCO, HPQ, INTC, JBL, KLAC...  
2     AFL, ADI, AMAT, BK, CSCO, HPQ, INTC, JBL, KLAC...  
3     AFL, ADI, AMAT, BK, CSCO, HPQ, INTC, JBL, KLAC...  
4     AFL, ADI, AMAT, BK, CSCO, HPQ, INTC, JBL, KLAC...  
...                                                 ...  
1541  CPB, INCY, LLY, VRTX, CPT, BIIB, CAH, BA, FICO, T  
1542  CPB, INCY, LLY, VRTX, CPT, BIIB, CAH, BA, FICO, T  
1543        CPB, INCY, LLY, CPT, BIIB, CAH, B

In [51]:
portfolio_df.tail(50)

,Date,Portfolio_Value,Held_Stocks
1496,2024-01-21,856056.530716,"CPB, INCY, LLY, VRTX, CPT, MNST, BIIB, CAH, BA..."
1497,2024-01-28,875196.597540,"CPB, INCY, LLY, VRTX, CPT, MNST, BIIB, CAH, BA..."
1498,2024-02-04,896113.537823,"CPB, INCY, LLY, VRTX, CPT, MNST, BIIB, CAH, BA..."
1499,2024-02-11,905488.128936,"CPB, INCY, LLY, VRTX, CPT, MNST, BIIB, CAH, BA..."
1500,2024-02-18,910217.028622,"CPB, INCY, LLY, VRTX, CPT, MNST, BIIB, CAH, BA..."
1501,2024-02-25,926672.100886,"CPB, INCY, LLY, VRTX, CPT, MNST, BIIB, CAH, BA..."
1502,2024-03-03,921980.464224,"CPB, INCY, LLY, VRTX, CPT, MNST, BIIB, CAH, BA..."
1503,2024-03-10,904598.909388,"CPB, INCY, LLY, VRTX, CPT, MNST, BIIB, CAH, BA..."
1504,2024-03-17,913882.093036,"CPB, INCY, LLY, VRTX, CPT, MNST, BIIB, CAH, BA..."
1505,2024-03-24,921125.196805,"CPB, INCY, LLY, VRTX, CPT, MNST, BIIB, CAH, BA..."


In [27]:
merged_df = stock.merge(sp500, left_on="Stock_Date", right_on="Date", how="left")

# Drop duplicate Date column (keeping Stock_Date)
merged_df = merged_df.drop(columns=["Date","Close","High", "Low","Volume"])


,Stock_Date,Stock_Close,Stock_High,Stock_Low,Stock_Open,Stock_Volume,Stock_20_week_high,Stock_20_week_ROC,Close,High,Low,Open,Volume,10_week_MA,Index_Filter
0,1995-01-01,0.312760,0.321138,0.282043,0.289489,1412812800,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1995-01-08,0.334169,0.357905,0.305313,0.309967,2665857600,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1995-01-15,0.317414,0.349993,0.316483,0.334169,1961400000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1995-01-22,0.296936,0.317414,0.290419,0.311829,1678667200,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1995-01-29,0.301590,0.313691,0.296936,0.298797,1124524800,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1561,2024-12-01,242.839996,244.630005,237.160004,237.270004,208286500,244.630005,8.505448,6090.270020,6099.970215,6033.390137,6040.109863,2.064771e+10,5892.583936,GREEN
1562,2024-12-08,248.130005,250.800003,241.750000,241.830002,192702600,250.800003,14.099154,6051.089844,6092.589844,6029.890137,6083.009766,2.013779e+10,5922.585938,GREEN
1563,2024-12-15,254.490005,255.000000,245.690002,247.990005,368202900,255.000000,16.012416,5930.850098,6085.189941,5832.299805,6063.790039,2.730266e+10,5934.167969,RED
1564,2024-12-22,255.589996,260.100006,253.059998,254.770004,133685900,260.100006,18.464366,5970.839844,6049.750000,5902.569824,5940.250000,1.141514e+10,5944.784961,GREEN


In [29]:
merged_df = merged_df.dropna()
merged_df

,Stock_Date,Stock_Close,Stock_High,Stock_Low,Stock_Open,Stock_Volume,Stock_20_week_high,Stock_20_week_ROC,Close,High,Low,Open,Volume,10_week_MA,Index_Filter
20,1995-05-21,0.318753,0.331354,0.315486,0.317353,1210630400,0.357905,1.916218,523.650024,531.909973,519.190002,519.190002,1.673100e+09,512.904004,GREEN
21,1995-05-28,0.315425,0.321042,0.307001,0.319169,648334400,0.349993,-5.609106,532.510010,536.909973,521.380005,523.650024,1.353120e+09,516.058005,GREEN
22,1995-06-04,0.325721,0.332273,0.315425,0.317297,1017206400,0.332273,2.617252,527.940002,537.729980,526.000000,532.510010,1.623250e+09,518.781006,GREEN
23,1995-06-11,0.328529,0.334145,0.324785,0.329465,638075200,0.334145,10.639765,539.830017,539.979980,527.940002,527.940002,1.737790e+09,522.122006,GREEN
24,1995-06-18,0.365032,0.375328,0.325721,0.328529,2539958400,0.375328,21.036164,549.710022,551.070007,539.830017,539.830017,1.846230e+09,526.170007,GREEN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1561,2024-12-01,242.839996,244.630005,237.160004,237.270004,208286500,244.630005,8.505448,6090.270020,6099.970215,6033.390137,6040.109863,2.064771e+10,5892.583936,GREEN
1562,2024-12-08,248.130005,250.800003,241.750000,241.830002,192702600,250.800003,14.099154,6051.089844,6092.589844,6029.890137,6083.009766,2.013779e+10,5922.585938,GREEN
1563,2024-12-15,254.490005,255.000000,245.690002,247.990005,368202900,255.000000,16.012416,5930.850098,6085.189941,5832.299805,6063.790039,2.730266e+10,5934.167969,RED
1564,2024-12-22,255.589996,260.100006,253.059998,254.770004,133685900,260.100006,18.464366,5970.839844,6049.750000,5902.569824,5940.250000,1.141514e+10,5944.784961,GREEN


In [40]:
merged_df

,Stock_Date,Stock_Close,Stock_High,Stock_Low,Stock_Open,Stock_Volume,Stock_20_week_high,Stock_20_week_ROC,Close,High,Low,Open,Volume,10_week_MA,Index_Filter,Buy_Signal
20,1995-05-21,0.318753,0.331354,0.315486,0.317353,1210630400,0.357905,1.916218,523.650024,531.909973,519.190002,519.190002,1.673100e+09,512.904004,GREEN,0
21,1995-05-28,0.315425,0.321042,0.307001,0.319169,648334400,0.349993,-5.609106,532.510010,536.909973,521.380005,523.650024,1.353120e+09,516.058005,GREEN,0
22,1995-06-04,0.325721,0.332273,0.315425,0.317297,1017206400,0.332273,2.617252,527.940002,537.729980,526.000000,532.510010,1.623250e+09,518.781006,GREEN,0
23,1995-06-11,0.328529,0.334145,0.324785,0.329465,638075200,0.334145,10.639765,539.830017,539.979980,527.940002,527.940002,1.737790e+09,522.122006,GREEN,0
24,1995-06-18,0.365032,0.375328,0.325721,0.328529,2539958400,0.375328,21.036164,549.710022,551.070007,539.830017,539.830017,1.846230e+09,526.170007,GREEN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1561,2024-12-01,242.839996,244.630005,237.160004,237.270004,208286500,244.630005,8.505448,6090.270020,6099.970215,6033.390137,6040.109863,2.064771e+10,5892.583936,GREEN,0
1562,2024-12-08,248.130005,250.800003,241.750000,241.830002,192702600,250.800003,14.099154,6051.089844,6092.589844,6029.890137,6083.009766,2.013779e+10,5922.585938,GREEN,0
1563,2024-12-15,254.490005,255.000000,245.690002,247.990005,368202900,255.000000,16.012416,5930.850098,6085.189941,5832.299805,6063.790039,2.730266e+10,5934.167969,RED,0
1564,2024-12-22,255.589996,260.100006,253.059998,254.770004,133685900,260.100006,18.464366,5970.839844,6049.750000,5902.569824,5940.250000,1.141514e+10,5944.784961,GREEN,0
